# **Google Bard**

In [ ]:

!pip install bardapi
from bardapi import BardCookies


cookie_dict = {
    "__Secure-1PSID": "cookies	", # open google bard in your broswer then inspect > Network > "refresh the page"  > in Network navitage to chat to get all these cookies
    "__Secure-1PAPISID": "cookies	",
    "__Secure-1PSIDCC": "cookies	",
    # Any cookie values you want to pass session object.
}
bard = BardCookies(cookie_dict=cookie_dict)



In [ ]:
import os
import csv
from bardapi import BardCookies
import time
image_folder_path = "/content/drive/MyDrive/adversial perception/Final Result/GPT/no entry"
csv_output_path = "/content/drive/MyDrive/adversial perception/Final Result/GPT/no entry/bard_results.csv"

bard = BardCookies(cookie_dict=cookie_dict)

with open(csv_output_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['Image Name', 'Predicted Value'])

    for image_filename in os.listdir(image_folder_path):
        time.sleep(5)
        if image_filename.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(image_folder_path, image_filename)
            with open(image_path, 'rb') as image_file:
                image_data = image_file.read()

                bard_answer = bard.ask_about_image("Is the traffic sign displayed a real-world traffic sign that has the same shape, color, pattern and text as real world traffic sign ? Answer with 'yes' or 'no'. 3 word answer please", image_data)


                predicted_value = bard_answer['content']
                csv_writer.writerow([image_filename, predicted_value])

print("CSV file created successfully.")



# ***GPT-4V***

In [ ]:
import base64
import requests
import os
import csv
import time

api_key = "XX"

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Directory
image_folder = "/content/drive/MyDrive/adversial perception/RQ3/Selected_Images"

csv_file = "/content/drive/MyDrive/adversial perception/RQ3/Selected_Images/GPT_results.csv"

with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Image Name', 'Predicted Value'])

    for image_name in os.listdir(image_folder):
        image_path = os.path.join(image_folder, image_name)

        if not os.path.isfile(image_path) or not image_path.endswith(('.png', '.jpg', '.jpeg')):
            continue

        base64_image = encode_image(image_path)

        headers = {
          "Content-Type": "application/json",
          "Authorization": f"Bearer {api_key}"
        }

        payload = {
          "model": "gpt-4-vision-preview",
          "messages": [
            {
              "role": "user",
              "content": [
                {
                  "type": "text",
                  "text": "Is the traffic sign displayed a real-world traffic sign that has the same shape, color, pattern and text as real world traffic sign ? Answer with 'yes' or 'no'. 3 word answer please"
                },
                {
                  "type": "image_url",
                  "image_url": {
                    "url": f"data:image/jpeg;base64,{base64_image}"
                  }
                }
              ]
            }
          ],
          "max_tokens": 50
        }

        time.sleep(5)

        response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

        data = response.json()

        content_value = data['choices'][0]['message']['content']

        writer.writerow([image_name, content_value])

# ***LLAVA***

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/LLaVA
%cd /content/LLaVA

!pip install -q transformers==4.36.2
!pip install ninja
!pip install flash-attn --no-build-isolation

!pip install -e .


In [ ]:
#after that modify cli.py file

import argparse
import os
import csv
import torch
from llava.constants import IMAGE_TOKEN_INDEX, DEFAULT_IMAGE_TOKEN, DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN
from llava.conversation import conv_templates, SeparatorStyle
from llava.model.builder import load_pretrained_model
from llava.utils import disable_torch_init
from llava.mm_utils import tokenizer_image_token, get_model_name_from_path, KeywordsStoppingCriteria
from PIL import Image
import requests
from io import BytesIO
from transformers import TextStreamer

def load_image(image_file):
    if image_file.startswith('http') or image_file.startswith('https'):
        response = requests.get(image_file)
        image = Image.open(BytesIO(response.content)).convert('RGB')
    else:
        image = Image.open(image_file).convert('RGB')
    return image

def process_image(image_file, model, tokenizer, image_processor, args, conv_mode, roles, csv_writer):
    try:
        image = load_image(image_file)
        image_tensor = image_processor.preprocess(image, return_tensors='pt')['pixel_values'].half().cuda()

        inp = """Is the traffic sign displayed a real-world traffic sign that has the same shape, color, pattern and text as real world traffic sign ? Answer with 'yes' or 'no'."""

        if model.config.mm_use_im_start_end:
            inp = DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_TOKEN + DEFAULT_IM_END_TOKEN + '\n' + inp
        else:
            inp = DEFAULT_IMAGE_TOKEN + '\n' + inp

        conv = conv_templates[conv_mode].copy()

        conv.append_message(roles[0], inp)
        conv.append_message(roles[1], None)
        prompt = conv.get_prompt()

        input_ids = tokenizer_image_token(prompt, tokenizer, IMAGE_TOKEN_INDEX, return_tensors='pt').unsqueeze(0).cuda()
        stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
        keywords = [stop_str]
        stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)
        streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

        with torch.inference_mode():
            output_ids = model.generate(
                input_ids,
                images=image_tensor,
                do_sample=True,
                temperature=args.temperature,
                max_new_tokens=args.max_new_tokens,
                streamer=streamer,
                use_cache=True,
                stopping_criteria=[stopping_criteria])

        outputs = tokenizer.decode(output_ids[0, input_ids.shape[1]:]).strip()

        if args.debug:
            print("\n", {"prompt": prompt, "outputs": outputs}, "\n")

        print(f"Processed {image_file}: {outputs}")

        csv_writer.writerow([os.path.basename(image_file), outputs])

    except IndexError as e:
        print(f"IndexError: {e}")
        print(f"Image file: {image_file}")

def main(args):
    disable_torch_init()
    model_name = get_model_name_from_path(args.model_path)
    tokenizer, model, image_processor, context_len = load_pretrained_model(args.model_path, args.model_base, model_name, args.load_8bit, args.load_4bit)

    if 'llama-2' in model_name.lower():
        conv_mode = "llava_llama_2"
    elif "v1" in model_name.lower():
        conv_mode = "llava_v1"
    elif "mpt" in model_name.lower():
        conv_mode = "mpt"
    else:
        conv_mode = "llava_v0"

    if args.conv_mode is not None and conv_mode != args.conv_mode:
        print(f'[WARNING] the auto inferred conversation mode is {conv_mode}, while `--conv-mode` is {args.conv_mode}, using {args.conv_mode}')
    else:
        args.conv_mode = conv_mode

    roles = ('user', 'assistant') if "mpt" in model_name.lower() else conv_templates[args.conv_mode].roles

    image_folder = '/LLaVA/non_ae_real/LLama_7b/03.StopSign'  # Replace with your folder path

    csv_file = open('/LLaVA/non_ae_real/LLama_7b/03.StopSign/stopsign_lama13.csv', 'w', newline='')
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['Image Name', 'Prediction'])

    all_files = os.listdir(image_folder)

    image_files = [os.path.join(image_folder, f) for f in all_files if f.lower().endswith(('.jpg', '.png'))]

    for image_file in image_files:
        process_image(image_file, model, tokenizer, image_processor, args, conv_mode, roles, csv_writer)

    csv_file.close()

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--model-path", type=str, default="facebook/opt-350m")
    parser.add_argument("--model-base", type=str, default=None)
    parser.add_argument("--num-gpus", type=int, default=1)
    parser.add_argument("--conv-mode", type=str, default=None)
    parser.add_argument("--temperature", type=float, default=0.2)
    parser.add_argument("--max-new-tokens", type=int, default=512)
    parser.add_argument("--load-8bit", action="store_true")
    parser.add_argument("--load-4bit", action="store_true")
    parser.add_argument("--debug", action="store_true")
    args = parser.parse_args()
    main(args)



In [1]:
# LLava13b
!python -m llava.serve.cli \
    --model-path 4bit/llava-v1.5-13b-3GB \
    --load-4bit

/usr/bin/python3: Error while finding module specification for 'llava.serve.cli' (ModuleNotFoundError: No module named 'llava')


In [ ]:
# LLava7b
!python -m llava.serve.cli \
    --model-path 4bit/llava-v1.5-7b-5GB \
    --load-4bit

# ***LLAVA C++ to fit Raspberry Pi and ZED box***

In [ ]:
!git clone https://github.com/ggerganov/llama.cpp
%cd llama.cpp

!mkdir build
!cd build
!cmake ..
!cmake --build . --config Release

In [ ]:
#modify the file  build/bin/run.sh

#!/bin/bash

CPP_PROGRAM="./llava-cli"
MODEL_ARG="-m ggml-model-q4_k.gguf" # avalable on hugging face https://huggingface.co/jartine/llava-v1.5-7B-GGUF/tree/main
MM_PROJ_ARG="--mmproj mmproj-model-f16.gguf" # avalable on hugging face https://huggingface.co/jartine/llava-v1.5-7B-GGUF/tree/main
IMAGE_ARG="--image"
PROMPT_ARG="-p"
DESCRIPTION="\"Is the traffic sign displayed a real-world traffic sign that has the same shape, color, pattern and text as real world traffic sign ? Answer with 'yes' or 'no'."
declare -a PNG_FILES

for file in /home/user/GPT/lane+detection/Clear_Faded_Lane/*.png; do
    PNG_FILES+=("$file")
done

for IMAGE_PATH in "${PNG_FILES[@]}"; do
    "$CPP_PROGRAM" $MODEL_ARG $MM_PROJ_ARG $PROMPT_ARG "$DESCRIPTION" $IMAGE_ARG "$IMAGE_PATH"
done
